<a href="https://colab.research.google.com/github/tomek-l/fire-detect-nn/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/tomek-l/fire/master/model.py

--2020-01-24 22:48:38--  https://raw.githubusercontent.com/tomek-l/fire/master/model.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2377 (2.3K) [text/plain]
Saving to: ‘model.py’

model.py            100%[===================>]   2.32K  --.-KB/s    in 0s      

2020-01-24 22:48:38 (63.8 MB/s) - ‘model.py’ saved [2377/2377]



In [0]:
BACKBONES = ['resnet18','resnet34','resnet50','resnet101', 'densenet121', 'mobilenet']

In [0]:
from model import Model, load_dataset, accuracy

In [0]:
from google.colab import drive
drive.mount('/gdrive')
!ls '/gdrive/My Drive/cmpe_lab276/FireVideo/binary_dataset/'

In [0]:
!cp '/gdrive/My Drive/cmpe_lab276/FireVideo/binary_dataset/' . -r

In [0]:
trainloader = load_dataset('./binary_dataset/train')
train_batch_list = list(trainloader)

In [0]:
validloader = load_dataset('./binary_dataset/validation', batch_size=16)
valid_batch_list = list(validloader)
len(valid_batch_list)

In [0]:
def average(x):
    return sum(x)/len(x)

In [0]:
import torch
device = torch.device("cuda:0")
for b in BACKBONES:

    import torch.optim as optim

    m = Model(backbone=b)
    m = m.to(device)

    criterion = torch.nn.BCELoss()
    optimizer = optim.Adam(m.parameters(), lr=1e-5, weight_decay=1e-5)

    for epoch in range(6):  # epochs

        running_loss = []
        running_acc = []

        # epoch training
        for i, data in enumerate(train_batch_list):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data[0].to(device)
            labels = data[1].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = m(inputs)
            loss = criterion(outputs[:,0], labels.type_as(outputs[:,0]))
            loss.backward()
            optimizer.step()

            acc = accuracy(outputs, labels)
            # print statistics
            
            running_loss.append(loss.item())
            running_acc.append(acc)

            if i%20 == 19: print(f'epoch: {epoch+1}, batch: {i}, loss: {average(running_loss)}, accuracy: {average(running_acc)}')
        
        valid_acc = []
        # epoch validation
        for i, data in enumerate(valid_batch_list):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data[0].to(device)
            labels = data[1].to(device)

            # could pehaps do:
            # for param in m.parameters():
            #     param.requires_grad = False

            outputs = m(inputs)
            valid_acc.append(accuracy(outputs, labels))
        va = round(average(valid_acc), 4)
        print(f'validation accuracy {va}')
        
        fname =  f'{b}-epoch-{epoch}-acc={va}.pt'
        print(f'Saved {fname}')
        torch.save(m, fname)
        

    print(f'Finished Training: {b}')